In [6]:
import pandas as pd
import ast

In [8]:
# Converting Excel files into dataframe

df_bangalore = pd.read_excel("C:\\Users\\navee\\Downloads\\bangalore_cars.xlsx")
df_delhi = pd.read_excel("C:\\Users\\navee\\Downloads\\delhi_cars.xlsx")
df_chennai = pd.read_excel("C:\\Users\\navee\\Downloads\\chennai_cars.xlsx")
df_hyderabad = pd.read_excel("C:\\Users\\navee\\Downloads\\hyderabad_cars.xlsx")
df_jaipur = pd.read_excel("C:\\Users\\navee\\Downloads\\jaipur_cars.xlsx")
df_kolkata = pd.read_excel("C:\\Users\\navee\\Downloads\\kolkata_cars.xlsx")


Bangalore Car Data Processing

In [9]:
df_bangalore.head()

,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...


In [10]:
df_bangalore.drop(columns=["car_links"], inplace=True)
df_bangalore.head()

,new_car_detail,new_car_overview,new_car_feature,new_car_specs
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."


In [11]:
def process_state_car_data(df_state, city_name):
    # Convert strings to dicts
    df_state["new_car_detail_dict"] = df_state["new_car_detail"].apply(ast.literal_eval)
    df_state["new_car_overview_dict"] = df_state["new_car_overview"].apply(ast.literal_eval)
    df_state["new_car_feature_dict"] = df_state["new_car_feature"].apply(ast.literal_eval)
    df_state["new_car_specs_dict"] = df_state["new_car_specs"].apply(ast.literal_eval)

    # Flatten new_car_detail
    new_car_detail_df = pd.json_normalize(df_state["new_car_detail_dict"])

    # Flatten new_car_overview
    new_car_overview = pd.json_normalize(df_state["new_car_overview_dict"])
    new_car_overview.drop(columns=["bottomData"], inplace=True, axis=1)

    def extract_key_value_dicts(details):
        if isinstance(details, list):
            return {df['key']: df['value'] for df in details}
        else:
            return {}

    new_car_overview_flat = new_car_overview['top'].apply(extract_key_value_dicts)
    new_car_overview_df = pd.json_normalize(new_car_overview_flat)

    # Flatten new_car_feature
    new_car_feature = pd.json_normalize(df_state["new_car_feature_dict"])
    new_car_feature.drop(columns=["commonIcon"], inplace=True, axis=1)

    def extract_key_value_dicts_indexed(details):
        if isinstance(details, list):
            return {i: df['value'] for i, df in enumerate(details)}
        else:
            return {}

    new_car_feature_top = new_car_feature["top"].apply(extract_key_value_dicts_indexed)
    new_car_feature_top_df = pd.json_normalize(new_car_feature_top)

    new_car_feature_data = pd.json_normalize(new_car_feature["data"])

    # Extract lists from feature data parts
    def extract_feature_list(data_part):
        df_list = pd.json_normalize(data_part)
        feature_list = df_list["list"].apply(extract_key_value_dicts_indexed)
        return pd.json_normalize(feature_list)

    new_car_feature_list_1 = extract_feature_list(new_car_feature_data[0])
    new_car_feature_list_2 = extract_feature_list(new_car_feature_data[1])
    new_car_feature_list_3 = extract_feature_list(new_car_feature_data[2])
    new_car_feature_list_4 = extract_feature_list(new_car_feature_data[3])
    new_car_feature_list_5 = extract_feature_list(new_car_feature_data[4])

    # Flatten new_car_specs
    new_car_specs = pd.json_normalize(df_state["new_car_specs_dict"])
    new_car_specs.drop(columns=["commonIcon"], inplace=True, axis=1)

    def extract_key_value_dicts_specs(details):
        if isinstance(details, list):
            return {df['key']: df['value'] for df in details}
        else:
            return {}

    new_car_specs_top = new_car_specs["top"].apply(extract_key_value_dicts_specs)
    new_car_specs_top_df = pd.json_normalize(new_car_specs_top)

    new_car_specs_data = pd.json_normalize(new_car_specs["data"])

    def extract_specs_list(data_part):
        df_list = pd.json_normalize(data_part)
        specs_list = df_list["list"].apply(extract_key_value_dicts_specs)
        return pd.json_normalize(specs_list)

    new_car_specs_list_1 = extract_specs_list(new_car_specs_data[0])
    new_car_specs_list_2 = extract_specs_list(new_car_specs_data[1])
    new_car_specs_list_3 = extract_specs_list(new_car_specs_data[2])

    # Combine all parts (you can adjust what to concat here)
    combined_df = pd.concat([
        new_car_detail_df,
        new_car_overview_df,
        new_car_specs_top_df,
        new_car_specs_list_1,
        new_car_specs_list_2,
        new_car_specs_list_3
    ], axis=1)

    # Add city column
    combined_df['City'] = city_name
    return combined_df

In [12]:
df_bangalore_processed = process_state_car_data(df_bangalore, "Bangalore")

In [13]:
df_bangalore_processed

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Alloy Wheel Size,City
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,...,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,Bangalore
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,...,5.3 metres,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,Bangalore
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,...,4.9 meters,Disc,Drum,150 kmph,14.3 Seconds,Tubeless,5,242-litres,14,Bangalore
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,...,4.7 metres,Disc,Drum,172km/hr,14.2 Seconds,"Tubeless,Radial",4,407-litres,14,Bangalore
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,5.2 meters,Ventilated Disc,Solid Disc,190 Kmph,12 Seconds,"Tubeless,Radial",5,353-litres,16,Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,0,Diesel,SUV,"1,56,039",Manual,2,2nd Owner,Toyota,Toyota Fortuner,2012,...,5.9 metres,Ventilated Disc,Drum,176 Kmph,9.6 Seconds,"Tubeless,Radial",5,296-litres,17,Bangalore
1477,0,Petrol,Sedan,"56,000",Manual,2,2nd Owner,Maruti,Maruti SX4,2008,...,5.3 m,Ventilated Disc,Drum,177 kmph,12.1 seconds,"Tubeless, Radial",4,505 litres,16,Bangalore
1478,0,Petrol,Hatchback,"42,000",Manual,2,2nd Owner,Honda,Honda Brio,2014,...,4.5 metres,Disc,Drum,164 Kmph,14.5 Seconds,"Tubeless,Radial",5,175-litres,NaN,Bangalore
1479,0,Diesel,Hatchback,"93,003",Manual,1,1st Owner,Hyundai,Hyundai i20,2018,...,5.2 metres,Disc,Drum,180 Kmph,11.9 Seconds,Tubeless,5,295-litres,16,Bangalore


Using above function to other states

In [14]:
df_delhi_processed = process_state_car_data(df_delhi, "Delhi")
df_chennai_processed = process_state_car_data(df_chennai, "Chennai")
df_hyderabad_processed = process_state_car_data(df_hyderabad, "Hyderabad")
df_jaipur_processed = process_state_car_data(df_jaipur, "Jaipur")
df_kolkata_processed = process_state_car_data(df_kolkata, "Kolkata")


In [15]:
df_hyderabad_processed

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Top Speed,Acceleration,City
0,0,Petrol,Hatchback,"20,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Polo,2022,...,4.9,Disc,Drum,"Tubeless,Radial",R16,5,280,NaN,NaN,Hyderabad
1,0,Petrol,Hatchback,"85,582",Manual,2,2nd Owner,Hyundai,Hyundai EON,2014,...,4.6 metres,Disc,Drum,Tubeless,NaN,5,215-litres,135 Kmph,19 Seconds,Hyderabad
2,0,Petrol,SUV,"10,000",Manual,1,1st Owner,Hyundai,Hyundai Venue,2023,...,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,NaN,NaN,NaN,Hyderabad
3,0,Petrol,Hatchback,"50,750",Manual,1,1st Owner,Maruti,Maruti Baleno,2017,...,4.9 meters,Disc,Drum,"Tubeless,Radial",16,5,339-litres,180 Kmph,12.36 seconds,Hyderabad
4,0,Petrol,SUV,"10,000",Automatic,1,1st Owner,Mahindra,Mahindra Thar,2022,...,NaN,Disc,Drum,"Radial, Tubeless",18,3,NaN,NaN,NaN,Hyderabad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Mahindra,Mahindra XUV500,2013,...,5.6 metres,Disc & Caliper Type,Disc & Caliper Type,"Tubeless,Radial",17,5,NaN,175 kmph,12.5 Seconds,Hyderabad
1479,0,Diesel,Sedan,"50,000",Automatic,1,1st Owner,Jaguar,Jaguar XF,2014,...,5.75 metres,Disc,Disc,"Tubeless,Radial",20,4,540-litres,250 Kmph,6.4 Seconds,Hyderabad
1480,0,Diesel,Sedan,"1,20,000",Automatic,3,3rd Owner,BMW,BMW 5 Series,2018,...,5.6,Ventilated Disc,Ventilated Disc,"Tubeless, Runflat",R18,4,520-litres,235kmph,7.5,Hyderabad
1481,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Tata,Tata Nexon,2019,...,5.1m,Disc,Drum,Tubeless Radial Tyres,NaN,5,350,154.19 kmph,13.25 Seconds,Hyderabad


In [16]:
df_jaipur_processed

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,City
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,...,5.2 metres,Disc,Drum,180 Kmph,11.9 Seconds,Tubeless,16,5,295-litres,Jaipur
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,...,4.8,Disc,Drum,NaN,NaN,"Radial, Tubeless",NaN,5,268,Jaipur
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,...,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",NaN,5,235-litres,Jaipur
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,...,4.8 metres,Disc,Drum,165 Kmph,12.9 Seconds,Tubeless,NaN,5,256,Jaipur
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,...,4.6 metres,Ventilated Disc,Drum,152 kmph,18.6 Seconds,Tubeless Tyres,NaN,5,180-liters,Jaipur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,...,4.85metres,Disc,Drum,NaN,NaN,"Tubeless,Radial",14,5,170-litres,Jaipur
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,...,4.6m,Disc,Drum,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,Jaipur
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,...,NaN,Disc,Drum,NaN,NaN,"Tubeless, Radial",16,5,405,Jaipur
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,...,4.6m,Disc,Drum,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,Jaipur


In [17]:
df_kolkata_processed

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Acceleration,Top Speed,City
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,...,5.5 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,NaN,NaN,NaN,Kolkata
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,...,4.7m,Disc,Drum,NaN,NaN,5,222,NaN,NaN,Kolkata
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,...,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,405,NaN,NaN,Kolkata
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,...,5.20 m,Disc,Drum,"Tubeless,Radial",14,5,295 Lit,12.96 Sec,NaN,Kolkata
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,...,5.9 metres,Ventilated Disc,Drum,"Tubeless,Radial",16,5,460-litres,8.2 Seconds,212 Kmph,Kolkata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,...,NaN,Ventilated Disc,Drum,"Tubeless, Radial",NaN,5,313,NaN,NaN,Kolkata
1377,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,...,4.6 metres,Solid Disc,Drum,Tubeless Tyres,NaN,5,177-litres,19 Seconds,140 kmph,Kolkata
1378,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,...,5.42 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,475-litres,8.8 Seconds,230km/hr,Kolkata
1379,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,...,4.7 metres,Ventilated Disc,Drum,"Tubeless,Radial",14,5,236-liters,15 Seconds,156 Kmph,Kolkata


In [18]:
df_bangalore_processed.to_csv("bangalore_cars_final.csv", index=False)
df_delhi_processed.to_csv("delhi_cars_final.csv", index=False)
df_chennai_processed.to_csv("chennai_cars_final.csv", index=False)
df_hyderabad_processed.to_csv("hyderabad_cars_final.csv", index=False)
df_jaipur_processed.to_csv("jaipur_cars_final.csv", index=False)
df_kolkata_processed.to_csv("kolkata_cars_final.csv", index=False)


In [19]:
# Load city-wise car data CSV files into dataframes 
df_banglore_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\bangalore_cars_final.csv")
df_chennai_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\chennai_cars_final.csv")
df_delhi_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\delhi_cars_final.csv")
df_hyderabad_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\hyderabad_cars_final.csv")
df_jaipur_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\jaipur_cars_final.csv")
df_kolkata_cars = pd.read_csv(r"C:\Users\navee\Desktop\savi\capstone3\kolkata_cars_final.csv")

all_cities_car_data = [
    df_banglore_cars,
    df_chennai_cars,
    df_delhi_cars,
    df_hyderabad_cars,
    df_jaipur_cars,
    df_kolkata_cars
]

Merging all datasets

In [20]:

# List of DataFrames from different cities
all_cities_car_data = [df_banglore_cars, df_chennai_cars, df_delhi_cars, df_hyderabad_cars, df_jaipur_cars, df_kolkata_cars]

# Function to rename duplicate columns by adding _1, _2, etc.
def rename_duplicates(df):
    cols = list(df.columns)
    seen = {}
    new_cols = []
    for col in cols:
        if col not in seen:
            seen[col] = 0
            new_cols.append(col)
        else:
            seen[col] += 1
            new_cols.append(f"{col}_{seen[col]}")
    df.columns = new_cols
    return df

# Rename duplicates in each DataFrame
all_cities_car_data = [rename_duplicates(df) for df in all_cities_car_data]

# Find all unique columns from all DataFrames
all_columns = set()
for df in all_cities_car_data:
    all_columns.update(df.columns)

# Add missing columns with NaN to each DataFrame
for df in all_cities_car_data:
    for col in all_columns:
        if col not in df.columns:
            df[col] = pd.NA

# Reset index in each DataFrame to avoid index issues
all_cities_car_data = [df.reset_index(drop=True) for df in all_cities_car_data]

# Combine all DataFrames into one
combined_df = pd.concat(all_cities_car_data, ignore_index=True)

print(combined_df.shape)
combined_df.head()


(8369, 88)


C:\Users\navee\AppData\Local\Temp\ipykernel_1068\2595090178.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_cities_car_data, ignore_index=True)


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Cargo Volumn,Alloy Wheel Size.2,City,Drive Type.1,Turning Radius.1,Gear Box.1,Acceleration.1,Top Speed.1,Front Brake Type.1,Rear Brake Type.1
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,...,235-litres,NaN,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,...,352-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,...,242-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,...,407-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,353-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
combined_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Cargo Volumn,Alloy Wheel Size.2,City,Drive Type.1,Turning Radius.1,Gear Box.1,Acceleration.1,Top Speed.1,Front Brake Type.1,Rear Brake Type.1
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,...,235-litres,NaN,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,...,352-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,...,242-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,...,407-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,353-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,...,313,NaN,Kolkata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8365,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,...,177-litres,NaN,Kolkata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8366,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,...,475-litres,NaN,Kolkata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8367,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,...,236-liters,NaN,Kolkata,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Droping columns with missing values as it is not needed for usage

In [65]:
combined_df.isnull().sum()

it                       0
ft                       0
bt                       4
km                       0
transmission             0
                      ... 
Turning Radius.1      6277
Rear Brake Type.1     5796
Gear Box.1            5795
Front Brake Type.1    5796
Acceleration.1        6822
Length: 88, dtype: int64

In [22]:
combined_df.columns

Index(['it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 'oem',
       'model', 'modelYear', 'centralVariantId', 'variantName', 'price',
       'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl',
       'trendingText.heading', 'trendingText.desc', 'Registration Year',
       'Insurance Validity', 'Fuel Type', 'Seats', 'Kms Driven', 'RTO',
       'Ownership', 'Engine Displacement', 'Transmission',
       'Year of Manufacture', 'Mileage', 'Engine', 'Max Power', 'Torque',
       'Seats.1', 'Wheel Size', 'Color', 'Engine Type', 'Displacement',
       'Max Power.1', 'Max Torque', 'No of Cylinder', 'Values per Cylinder',
       'Value Configuration', 'Fuel Suppy System', 'BoreX Stroke',
       'Compression Ratio', 'Turbo Charger', 'Super Charger',
       'Seating Capacity', 'Steering Type', 'Tyre Type', 'Alloy Wheel Size',
       'No Door Numbers', 'Length', 'Width', 'Height', 'Wheel Base',
       'Front Tread', 'Rear Tread', 'Kerb Weight', 'Gross Weight',
      

In [23]:
combined_df.to_csv("combined_datas.csv",index=False)


In [24]:
combined_df.isnull().sum()

it                       0
ft                       0
bt                       4
km                       0
transmission             0
                      ... 
Gear Box.1            5795
Acceleration.1        6822
Top Speed.1           6900
Front Brake Type.1    5796
Rear Brake Type.1     5796
Length: 88, dtype: int64

In [25]:
# List of columns you want to keep (correcting case and spelling to match your df columns)
cols_to_keep = [
    'it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 'oem',
    'model', 'modelYear', 'price', 'Insurance Validity', 'Kms Driven',
    'Mileage', 'Engine', 'Seats.1', 'Color', 'Gear Box', 'Drive Type', 'City'
]

# Select only these columns from combined_df
combined_df = combined_df[cols_to_keep]



In [26]:
combined_df.isnull().sum()

it                       0
ft                       0
bt                       4
km                       0
transmission             0
ownerNo                  0
owner                    0
oem                      0
model                    0
modelYear                0
price                    0
Insurance Validity       4
Kms Driven               2
Mileage                287
Engine                   4
Seats.1                  6
Color                    3
Gear Box              2680
Drive Type            3714
City                     0
dtype: int64

In [27]:
combined_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,price,Insurance Validity,Kms Driven,Mileage,Engine,Seats.1,Color,Gear Box,Drive Type,City
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,₹ 4 Lakh,Third Party insurance,"1,20,000 Kms",23.1 kmpl,998 CC,5.0,White,5 Speed,FWD,Bangalore
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,₹ 8.11 Lakh,Comprehensive,"32,706 Kms",17 kmpl,1497 CC,5.0,White,5 Speed,FWD,Bangalore
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,₹ 5.85 Lakh,Comprehensive,"11,949 Kms",23.84 kmpl,1199 CC,5.0,Red,5 Speed,FWD,Bangalore
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,₹ 4.62 Lakh,Comprehensive,"17,794 Kms",19.1 kmpl,1197 CC,5.0,Others,5 Speed,FWD,Bangalore
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,₹ 7.90 Lakh,Third Party insurance,"60,000 Kms",23.65 kmpl,1248 CC,5.0,Gray,5 Speed,FWD,Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,₹ 5.10 Lakh,Third Party insurance,"10,000 Kms",25.24 kmpl,998 CC,5.0,Others,5-Speed,NaN,Kolkata
8365,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,₹ 1.80 Lakh,Third Party insurance,"1,20,000 Kms",22.74 kmpl,796 CC,5.0,Others,5 Speed,FWD,Kolkata
8366,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,₹ 5.50 Lakh,Third Party insurance,"50,000 Kms",11.74 kmpl,1796 CC,5.0,Others,7 Speed,RWD,Kolkata
8367,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,₹ 1.40 Lakh,Third Party insurance,"40,000 Kms",18.5 kmpl,1197 CC,5.0,Others,5 Speed,FWD,Kolkata


In [28]:
combined_df.to_csv("combined_datas_final.csv",index=False)